## 1. 🔧 Setup

First, let's load our environment and set up LangChain with modern 2025 imports.

In [ ]:
import os
from dotenv import load_dotenv

# Modern LangChain imports (2025 standard)
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

# Load API key from .env file
load_dotenv()

# Verify API key is loaded
if os.getenv("OPENAI_API_KEY"):
    print("✅ OPENAI_API_KEY loaded successfully!")
else:
    print("❌ ERROR: OPENAI_API_KEY not found. Please check your .env file.")

# Initialize the model
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
print(f"🤖 Model initialized: {llm.model_name}")

---

## 2. 🔴 The Problem: LLMs Have No Memory

Let's first demonstrate that LLMs are **stateless** — they don't remember previous messages by default.

In [ ]:
# ===========================================
# DEMONSTRATION: No Memory Without History
# ===========================================

print("🔴 WITHOUT MEMORY MANAGEMENT")
print("=" * 50)

# First message
response1 = llm.invoke([HumanMessage(content="My name is Alice.")])
print(f"👤 User: My name is Alice.")
print(f"🤖 AI: {response1.content}")
print()

# Second message - SEPARATE call, no history!
response2 = llm.invoke([HumanMessage(content="What is my name?")])
print(f"👤 User: What is my name?")
print(f"🤖 AI: {response2.content}")
print()
print("❌ The AI doesn't know our name because each call is independent!")

---

## 3. 🟢 The Solution: LangChain Memory

LangChain provides `InMemoryChatMessageHistory` and `RunnableWithMessageHistory` to automatically manage conversation history.

### How It Works

1. **ChatMessageHistory**: Stores the conversation messages
2. **RunnableWithMessageHistory**: Wraps your chain and automatically:
   - Loads previous messages before each call
   - Saves new messages after each call
3. **Session IDs**: Allow multiple separate conversations

In [ ]:
# ===========================================
# SETUP: Memory-Enabled Chain
# ===========================================

# Step 1: Create a prompt template with a placeholder for chat history
prompt = ChatPromptTemplate.from_messages([
    SystemMessage(content="You are a helpful, friendly AI assistant. Remember details the user tells you."),
    MessagesPlaceholder(variable_name="history"),  # <-- This is where history goes!
    ("human", "{input}")
])

# Step 2: Create the chain (prompt | model)
chain = prompt | llm

# Step 3: Create a store for session histories
# Each session_id gets its own separate conversation history
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """
    Get or create a chat history for a given session.
    
    This function is called by RunnableWithMessageHistory to:
    - Load existing history before a call
    - Save new messages after a call
    """
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# Step 4: Wrap the chain with memory management
chain_with_memory = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

print("✅ Memory-enabled chain created!")

In [ ]:
# ===========================================
# DEMONSTRATION: Memory in Action!
# ===========================================

print("🟢 WITH LANGCHAIN MEMORY")
print("=" * 50)

# Configuration for this session
config = {"configurable": {"session_id": "alice_session"}}

# First message: Tell the AI our name
response1 = chain_with_memory.invoke(
    {"input": "My name is Alice."},
    config=config
)
print(f"👤 User: My name is Alice.")
print(f"🤖 AI: {response1.content}")
print()

In [ ]:
# Second message: Ask the AI our name
# The AI should remember because of the memory management!

response2 = chain_with_memory.invoke(
    {"input": "What is my name?"},
    config=config  # Same session_id = same conversation
)
print(f"👤 User: What is my name?")
print(f"🤖 AI: {response2.content}")
print()
print("✅ The AI remembers our name because LangChain manages the history!")

In [ ]:
# Let's continue the conversation with more context

response3 = chain_with_memory.invoke(
    {"input": "I'm learning about AI. I work as a software engineer."},
    config=config
)
print(f"👤 User: I'm learning about AI. I work as a software engineer.")
print(f"🤖 AI: {response3.content}")
print()

In [ ]:
# Test if it remembers BOTH pieces of information

response4 = chain_with_memory.invoke(
    {"input": "What do you know about me so far?"},
    config=config
)
print(f"👤 User: What do you know about me so far?")
print(f"🤖 AI: {response4.content}")
print()
print("🎉 The AI remembers everything from the conversation!")

---

## 4. 📜 Inspecting the Stored History

Let's peek inside the memory store to see what LangChain is actually storing.

In [ ]:
# ===========================================
# INSPECT: What's in the memory store?
# ===========================================

print("📜 CONVERSATION HISTORY STORED BY LANGCHAIN")
print("=" * 50)

# Get the history for our session
history = get_session_history("alice_session")

# Print each message
for i, message in enumerate(history.messages, 1):
    role = type(message).__name__.replace("Message", "")
    content = message.content[:80] + "..." if len(message.content) > 80 else message.content
    print(f"{i}. [{role}]: {content}")

print()
print(f"📊 Total messages stored: {len(history.messages)}")

---

## 5. 🔀 Multiple Sessions (Separate Conversations)

The `session_id` allows you to have multiple independent conversations. This is useful for:
- Multi-user applications
- Separate conversation threads
- Testing different scenarios

In [ ]:
# ===========================================
# MULTIPLE SESSIONS DEMO
# ===========================================

print("🔀 MULTIPLE INDEPENDENT SESSIONS")
print("=" * 50)

# Session 1: Bob's conversation
bob_config = {"configurable": {"session_id": "bob_session"}}

response_bob = chain_with_memory.invoke(
    {"input": "Hi! My name is Bob and I love pizza."},
    config=bob_config
)
print(f"🔵 Bob's Session:")
print(f"   👤 Bob: Hi! My name is Bob and I love pizza.")
print(f"   🤖 AI: {response_bob.content[:100]}...")
print()

# Session 2: Charlie's conversation (separate!)
charlie_config = {"configurable": {"session_id": "charlie_session"}}

response_charlie = chain_with_memory.invoke(
    {"input": "Hello! I'm Charlie and I'm a musician."},
    config=charlie_config
)
print(f"🟡 Charlie's Session:")
print(f"   👤 Charlie: Hello! I'm Charlie and I'm a musician.")
print(f"   🤖 AI: {response_charlie.content[:100]}...")
print()

In [ ]:
# Verify sessions are independent

print("🔍 VERIFYING SESSION INDEPENDENCE")
print("=" * 50)

# Ask Bob's session about the user
bob_check = chain_with_memory.invoke(
    {"input": "What's my name and what do I like?"},
    config=bob_config
)
print(f"🔵 Bob's Session - 'What's my name and what do I like?'")
print(f"   🤖 AI: {bob_check.content}")
print()

# Ask Charlie's session about the user
charlie_check = chain_with_memory.invoke(
    {"input": "What's my name and what do I do?"},
    config=charlie_config
)
print(f"🟡 Charlie's Session - 'What's my name and what do I do?'")
print(f"   🤖 AI: {charlie_check.content}")
print()

print("✅ Each session remembers only its own conversation!")

---

## 📝 Summary

### What You Learned Today

✅ **The Problem**
- LLMs are stateless — they don't remember previous messages
- Without memory management, each API call is independent

✅ **Manual Solution** (from `simple_chatbot.py`)
- Maintain a `conversation_history` list
- Append every message (user + AI) to the list
- Send the entire history with each request

✅ **LangChain Solution** (this notebook)
- `InMemoryChatMessageHistory`: Stores messages in memory
- `RunnableWithMessageHistory`: Automatically loads/saves history
- `session_id`: Enables multiple independent conversations

### Key Components

```python
# 1. Create history store
store = {}
def get_session_history(session_id):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

# 2. Wrap chain with memory
chain_with_memory = RunnableWithMessageHistory(
    chain, get_session_history,
    input_messages_key="input",
    history_messages_key="history"
)

# 3. Use with session config
config = {"configurable": {"session_id": "my_session"}}
response = chain_with_memory.invoke({"input": "Hello!"}, config=config)
```

---

### 🚀 Next Steps

In the next module, we'll explore **RAG (Retrieval Augmented Generation)** — giving your AI access to external knowledge!

Happy building! 🎉